In [1]:
import common

import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from dotenv import load_dotenv
from datetime import datetime
from zoneinfo import ZoneInfo
import wandb

In [2]:
common.wandb_login_init('SKF_densenet169.tv_in1k')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kerneld82 (kerneld). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc


train_time = SKF_densenet169.tv_in1k


In [3]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = 'datasets_fin/'

# model config
model_name = 'densenet169.tv_in1k' # 'resnet50' 'efficientnet-b0', ...

# training config
trn_img_size = 224
tst_img_size = 224
LR = 1e-3
EPOCHS = 20
FOLDS = 2
BATCH_SIZE = 32
num_workers = 12
augment_ratio = 200

wandb.config.update({
    "learning_rate": LR,
    "architecture": model_name,
    "dataset": "custom-dataset",
    "epochs": EPOCHS,
    "folds": FOLDS,
    "batch_size": BATCH_SIZE,
    "train_image_size": trn_img_size,
    "test_image_size": tst_img_size,
    "num_workers": num_workers,
    'augment_ratio' : augment_ratio,
})

In [4]:
# 시드를 고정합니다.
SEED = 42

common.set_seed(SEED)

In [5]:
device

device(type='cuda')

### Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [6]:
# 학습과 검증에 사용할 폴드별 csv 파일들을 먼저 만들기.
# 나중에 이어서 학습하고 싶을때 사용하기 위해서.
common.generate_fold_train_valid_csv_files(SEED, FOLDS)

Fold 1/2, train_idx: <class 'numpy.ndarray'> 867, <class 'numpy.ndarray'> 867
Fold 2/2, train_idx: <class 'numpy.ndarray'> 867, <class 'numpy.ndarray'> 867


In [7]:
for fold in range(FOLDS):
    fold += 1
    print(f"Fold {fold}/{FOLDS}")
    
    supplies = common.get_supplies_for_train_and_valid_with_fold(seed =SEED, 
                                                                 model_name = model_name, 
                                                                 lr = LR,
                                                                 batch_size = BATCH_SIZE, 
                                                                 num_workers = num_workers, 
                                                                 fold = fold, 
                                                                 folds = FOLDS, 
                                                                 augment_ratio = augment_ratio, 
                                                                 trn_img_size = trn_img_size, 
                                                                 tst_img_size = tst_img_size, 
                                                                 device = device)
    
    common.train_with_start_end_epoch(seed = SEED, 
                                      tst_img_size = tst_img_size,
                                      batch_size = BATCH_SIZE,
                                      start_epoch_inclusive = 1, 
                                      end_epoch_exclusive = EPOCHS + 1, 
                                      augment_ratio = augment_ratio,
                                      trn_loader = supplies['trn_loader'], 
                                      val_loader = supplies['val_loader'], 
                                      model = supplies['model'], 
                                      model_name = model_name, 
                                      optimizer = supplies['optimizer'], 
                                      loss_fn = supplies['loss_fn'], 
                                      device = device, 
                                      is_save_model_checkpoint = True, 
                                      is_evaluate_train_valid = True,
                                      fold = fold,
                                      folds = FOLDS)
    

Fold 1/2


model.safetensors:   0%|          | 0.00/57.4M [00:00<?, ?B/s]


epoch: 1
다음 epoch 학습을 중단 예약하고 싶으면, 빈 BREAK 파일을 생성하세요.


Loss: 0.0629: 100%|██████████| 5419/5419 [09:21<00:00,  9.65it/s]


train_loss: 0.2241
train_acc: 0.9210
train_f1: 0.9210



...


final_valid_loss: 0.5566798395261036
final_valid_accuracy: 0.9238754325259516
final_valid_f1: 0.923020475714169
valid's error preds count: 66/867, {7: 31, 3: 12, 11: 3, 13: 3, 14: 14, 4: 1, 0: 1, 10: 1}
Model checkpoint saved. filename: cp-densenet169.tv_in1k_sd_42_epc_7_aug_200_vl_0.5567_va_0.9239_vf1_0.9230_fold_2_folds_2.pt
5초 동안 잠시 sleep 합니다. 다음 epoch 학습을 중단 예약하고 싶으면, 빈 BREAK 파일을 생성하세요.
BREAK 파일이 있는지 확인합니다.

epoch: 8
다음 epoch 학습을 중단 예약하고 싶으면, 빈 BREAK 파일을 생성하세요.


Loss: 0.0000: 100%|██████████| 5419/5419 [09:25<00:00,  9.59it/s]


train_loss: 0.0161
train_acc: 0.9946
train_f1: 0.9946



Evaluating: 100%|██████████| 28/28 [00:04<00:00,  6.73it/s]



final_valid_loss: 0.702215315641037
final_valid_accuracy: 0.9284890426758939
final_valid_f1: 0.9281185070205868
valid's error preds count: 62/867, {7: 27, 3: 15, 11: 3, 14: 12, 4: 2, 13: 1, 10: 1, 0: 1}
Model checkpoint saved. filename: cp-densenet169.tv_in1k_sd_42_epc_8_aug_200_vl_0.7022_va_0.9285_vf1_0.9281_fold_2_folds_2.pt
5초 동안 잠시 sleep 합니다. 다음 epoch 학습을 중단 예약하고 싶으면, 빈 BREAK 파일을 생성하세요.
BREAK 파일이 있는지 확인합니다.
BREAK 파일이 존재합니다. 학습을 중단합니다.


In [8]:
# wandb 실행 종료
wandb.finish()

final_valid_accuracy,▆█▆▇▆▆▇▁▆▅▅▅▇▅▆
final_valid_f1,▅█▆▆▆▅▆▁▆▄▄▅▇▄▅
final_valid_loss,▁▂▁▃▆▆▆▃▂▄▅▅▅▅█
train_acc,▁▆▇▇███▁▆▇▇████
train_f1,▁▆▇▇███▁▆▇▇████
train_loss,█▃▂▂▁▁▁█▃▂▂▁▁▁▁
final_valid_accuracy,0.92849
final_valid_f1,0.92812
final_valid_loss,0.70222
train_acc,0.99462
train_f1,0.9946


In [9]:
import sys
sys.exit('아래 셀은 수동으로 실행하기 위해서 여기서 실행 멈춤.')

SystemExit: 아래 셀은 수동으로 실행하기 위해서 여기서 실행 멈춤.

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# 수동으로 특정 체크포인트부터 이어서 학습하기 위한 부분


In [ ]:
# checkpoint = common.load_model_checkpoint("checkpoint-resnet34_seed_42_epoch_0_isFull_False.pt", model, optimizer, device)

In [ ]:
# next_epoch = checkpoint['epoch'] + 1

# common.train_with_start_end_epoch(seed = checkpoint['seed'],
#                            tst_img_size = checkpoint['tst_img_size'],
#                            batch_size = checkpoint['batch_size'],
#                            start_epoch_inclusive = next_epoch, 
#                            end_epoch_exclusive = next_epoch + 2, 
#                            augment_ratio = augment_ratio,
#                            trn_loader = trn_loader,
#                            val_loader = val_loader,
#                            model = model,
#                            model_name = model_name,
#                            optimizer = optimizer,
#                            loss_fn = loss_fn,
#                            device = device,
#                            is_save_model_checkpoint = True,
#                            is_evaluate_train_valid = True)